In [1]:
import pickle
import pandas as pd
from load_team_data import full_kenpom_pipeline, read_unplayed_tournament
from load_player_data import load_player_data
from simulate_tournament import simulate_n_tournaments

In [ ]:
year = 2024
kenpom_ratings_df = full_kenpom_pipeline(year)
matchups_dict = read_unplayed_tournament(year)
player = load_player_data(year, matchups_dict)

Scraping KenPom data for 2024...


/Users/zsg/projects/mm25-copy/scripts/load_team_data.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  kenpom_df = pd.read_html(str(table))[0]


Loading player data for team link: /cbb/schools/connecticut/men/2024.html
Loading player data for team link: /cbb/schools/stetson/men/2024.html
{'UConn': {'Tristen Newton': {'ppg': 15.1, 'running_total_simulated': 0, 'link': '/cbb/players/tristen-newton-1.html', 'ground_truth_total': np.float64(87.0)}, 'Cam Spencer': {'ppg': 14.3, 'running_total_simulated': 0, 'link': '/cbb/players/cam-spencer-1.html', 'ground_truth_total': np.float64(80.0)}, 'Alex Karaban': {'ppg': 13.3, 'running_total_simulated': 0, 'link': '/cbb/players/alex-karaban-1.html', 'ground_truth_total': np.float64(58.0)}, 'Donovan Clingan': {'ppg': 13.0, 'running_total_simulated': 0, 'link': '/cbb/players/donovan-clingan-1.html', 'ground_truth_total': np.float64(92.0)}, 'Stephon Castle': {'ppg': 11.1, 'running_total_simulated': 0, 'link': '/cbb/players/stephon-castle-2.html', 'ground_truth_total': np.float64(75.0)}, 'Hassan Diarra': {'ppg': 6.1, 'running_total_simulated': 0, 'link': '/cbb/players/hassan-diarra-1.html', 'gr

In [ ]:

probs, sims = simulate_n_tournaments(matchups_dict, player, kenpom_ratings_df, N=1000)

# Convert the probabilities to a DataFrame for better readability
df = pd.DataFrame(probs.items(), columns=['Team', 'Probability'])
df = df.sort_values(by='Probability', ascending=False)
df.reset_index(drop=True, inplace=True)

In [ ]:
def convert_player_data(old_file):
    with open(old_file, 'rb') as f:
            player_data = pickle.load(f)

    new = {}
    for team in player_data:
        new[team] = {}
        for player in player_data[team]:
            new[team][player] = {'avg': player_data[team][player], 'running_total': 0}

    with open(f'NEW_player_data_2024.pkl', 'wb') as f:
            pickle.dump(new, f)
        

# print(main())
# convert_player_data('player_data_2024.pkl')
# print(load_player_data(2024, {}))


In [ ]:
target_players = {
    ("Purdue", "Zach Edey"): 1,
    ("Tennessee", "Dalton Knecht"): 1,
    ("Houston", "LJ Cryer"): 1,
    ("Clemson", "PJ Hall"): 2,
    ("Dayton", "DaRon Holmes II"): 2,
    ("Morehead State", "Riley Minix"): 3,
    ("Michigan State", "Tyson Walker"): 2,
    ("UNC", "RJ Davis"): 1,
    ("Florida", "Walter Clayton Jr."): 2,
    ("Florida", "Zyon Pullin"): 2,
    ("Clemson", "Joseph Girard III"): 2,
    ("Texas", "Max Abmas"): 2,
    ("Texas Tech", "Pop Isaacs"): 2,
    ("BYU", "Jaxson Robinson"): 2,
    ("Akron", "Enrique Freeman"): 3
}

global_sum_avg = 0
num_sims = len(sims)
print("Player Average Running Totals Over All Simulations:")

for (team, player), multiplier in target_players.items():
    total_sum = 0
    sim_count = 0
    for sim in sims:
        # Retrieve the player's running_total, if available in the simulation bookkeeping
        running_total = sim.players_bookkeeping.get(team, {}).get(player, {}).get("running_total")
        if running_total is not None:
            total_sum += running_total
            sim_count += 1
    if sim_count > 0:
        avg_total = total_sum / sim_count
        print(f"Team: {team:15s} | Player: {player:20s} | "
              f"Average Running Total: {avg_total:.2f} (Expected x{multiplier})")
        global_sum_avg += avg_total
    else:
        print(f"Team: {team:15s} | Player: {player:20s} | Total Not Found")

print("\nGlobal Sum of Averages for Target Players:", global_sum_avg)